# Exemplo de Modelo MPP otimizando por capacidade residual


$$Z = min_{(i,j) \in E} \left\{ z_{ij} \right\}$$

$$\mbox{Subject to:}$$

$$\sum_{i \in V}{x_{ir_k}^{kd}} - \sum_{i \in V}{x_{r_{k}i}^{kd}} = -1\quad {\forall k \in K \atop \forall d \in D^k}$$

$$\sum_{(i,d) \in E}{x_{id}^{kd}} - \sum_{(d,i) \in E}{x_{di}^{kd}} = 1 \quad {\forall k \in K \atop \forall d \in D^k}$$

$$\sum_{(i,j) \in E \atop j \neq r_k,d}{x_{ij}^{kd}} - \sum_{(i,j) \in E \atop j \neq r_k,d}{x_{ji}^{kd}} = 0 \quad {\forall k \in K \atop \forall d \in D^k}$$

$$x_{ij}^{kd} \leq y_{ij}^{k} \quad \forall k \in K, d \in D^k  \text{ e }  \forall (i,j) \in E$$

$$\sum_{d \in D^k}{x_{ij}^{kd}} - y_{ij}^{k} \geq 0 \quad \forall k \in K$$

$$b_{ij}  \geq \sum_{k \in K} y_{ij}^{k} t^k \quad \forall (i,j) \in E$$

$$b_{ij}  - \sum_{k \in K} y_{ij}^{k} t^k = z_{ij} \quad \forall (i,j) \in E$$

$$\sum_{(i,j) \in E} y_{ij}^{k} c_{ij} \leq B, \quad \forall k \in K$$

$$x_{ij}^{kd} \in \lbrace 0, 1 \rbrace , y_{ij}^{k} \in \lbrace 0, 1 \rbrace  {{\forall k \in K,  \forall d \in D^k} \atop \forall (i,j) \in E}$$




# Obtendo informações da rede e dos grupos

In [6]:
from classes import Network, MulticastGroup
from multicastpacking import MulticastPacking, solver
import reader

#instancen of the problem
file = "../../../MPP_instances/n30/b30_1.brite"

links = reader.get_network (file)
net = Network (links, nodes = 30)


mgroups = [MulticastGroup (g) for g in reader.get_groups (file) ]

problem = MulticastPacking (net, mgroups)

KSIZE = len(problem.groups)+1
NODES = net.nodes+1

# Criando as restrições de Fluxo

## Criando as variáveis do problema: $x_{ij}^{kd}, y_{ij}^{k}$

In [7]:
from gurobipy import *

m = Model ("MPP by Cost")

var_x = {}
var_y = {}

#variable x
for link in net.links:
    for k in xrange (1, KSIZE):
        for d in problem.groups[k-1].members:
            x=link[0],link[1],k,d,
            y=link[1],link[0],k,d,
            var_x[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
            var_x[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))
            
m.update ()

#variable y
for k in xrange (1, KSIZE):
    for link in net.links:
        x=link[0],link[1],k
        y=link[1],link[0],k
        var_y[x] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(x))
        var_y[y] = m.addVar(vtype=GRB.BINARY, obj=1, name=str(y))

m.update ()

## Fluxo 1:  

$\sum_{i \in V} x_{ij}^{kd} - \sum_{i \in V} x_{ji}^{kd} = -1 \quad \forall k \in K,\forall d \in D^k$ 

In [8]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow1',k,d
        m.addConstr (
            quicksum ( var_x[x] for x in tuplelist (var_x).select ('*',sk,k,d) )
            -
            quicksum ( var_x[x] for x in tuplelist (var_x).select (sk,'*',k,d) )
            == -1,
            name=str(_name)
        )

m.update ()

## Fluxo 2: 

$\sum_{(i,j) \in E,  j \notin [r_k, d]} x_{ij}^{kd} - \sum_{(i,j) \in E, j \notin [r_k, d]} x_{ji}^{kd} = 0 \quad \forall k \in K,\forall d \in D^k$ 

In [9]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for j in xrange(1,NODES):
            sk = problem.groups[k-1].source
            _name='flow2',k,d,j,       
            m.addConstr (
                quicksum(
                   var_x[x] for x in tuplelist (var_x).select ('*',j,k,d) 
                    if x[1] not in [sk, d]
                )
                -
                quicksum(
                    var_x[x] for x in tuplelist (var_x).select (j,'*',k,d) 
                    if x[0] not in [sk, d]
                )
                == 0,
                name=str(_name)
            )            

m.update ()


## Fluxo 3:  

$$\sum_{(d,i) \in E} x_{di}^{kd} - \sum_{(i,d) \in E} x_{id}^{kd} = 1 \quad \forall k \in K,\forall d \in D^k$$

In [10]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        sk = problem.groups[k-1].source
        _name='flow3',k,d
        m.addConstr (
            quicksum (
                var_x[x] for x in tuplelist (var_x).select ('*',d,k,d)
            )
            -
            quicksum (
                var_x[x] for x in tuplelist (var_x).select (d,'*',k,d)
            )
            == 1,
            name=str(_name)
        )
        
m.update ()

## Restrição que Força $y_{ij}^{k}$ ser igual 1 se a aresta $(i,j)$ é usada por alguma árvore

In [11]:
for k in xrange (1, KSIZE):
    for d in problem.groups[k-1].members:
        for link in net.links:
            x=link[0],link[1],k,d,
            y=link[0],link[1],k
            _name='r4',link[0],link[1],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )            
            x=link[1],link[0],k,d,
            y=link[1],link[0],k
            _name='r4',link[1],link[0],k,d
            m.addConstr ( var_x[x] <= var_y[y], 
                name=str(_name)
            )
m.update ()

## Restrição que evita nós folhas não terminais.

In [12]:
for k in xrange(1,KSIZE):
    for link in net.links:
        _name='out',link[0],link[1],k
        m.addConstr(
            quicksum(
                  var_x[x] for x in tuplelist (var_x).select (link[0],link[1],k,'*')              
            )
            -
            var_y[link[0],link[1],k]
            >=
            0,
            name=str(_name)
        )
        _name='out',link[1],link[0],k,
        
        m.addConstr(
            quicksum(
                  var_x[x] for x in tuplelist (var_x).select (link[1],link[0],k,'*')              
            )
            -
            var_y[link[1],link[0],k]
            >=
            0,
            name=str(_name)
        )
m.update()

In [47]:
print (m)

<gurobi.Model MIP instance MPP by Cost: 5432 constrs, 4440 vars, Parameter changes: LogFile=gurobi.log>


# Criando Variável para Capacidade Residual

In [15]:
Z = m.addVar(vtype=GRB.INTEGER, name="Z", obj=1)
m.update ()

## Forçando todos os links terem pelo menos a mesma capacidade Z 

$b_{ij} - \sum_{k \in K} y_{ij}^{k}t^{k} $

In [16]:
for k in xrange(1,KSIZE):
    for link in net.links:
        _name='cap',k,link[0],link[1]
        m.addConstr (
            net.links[link][1] - quicksum ( var_y[i] * problem.groups[ i[2]-1 ].traffic
                                           for i in tuplelist (var_y).select (link[0],link[1],'*') ) 
            >= 
            Z,
            name=str(_name)
        )
        _name='cap',k,link[1],link[0]
        m.addConstr (
            net.links[link][1] - quicksum ( var_y[i] * problem.groups[ i[2]-1 ].traffic
                                           for i in tuplelist (var_y).select (link[1],link[0],'*') ) 
            >= 
            Z,
            name=str(_name)
        )
m.update ()     

# Restrição de Hop: Atributo de QoS:
$$ \sum_{(i,j) \in E} x_{ij}^{k} <= HOP \quad \forall k \in K, \forall d \in D^k$$


In [17]:
HOP = 5
for k in xrange(1, KSIZE):
    for d in problem.groups[k-1].members:
        _name='hop',k,d
        m.addConstr(
            quicksum(
                 var_x[x] for x in tuplelist (var_x).select ('*','*',k,d)
            )
            <=
            HOP,
            name=str(_name)
        )
m.update ()

# Função Objetivo

In [18]:
m.setObjective (Z, GRB.MAXIMIZE)
m.update ()

In [19]:
m.optimize ()

Optimize a model with 6096 rows, 4441 columns and 27240 nonzeros
Coefficient statistics:
  Matrix range    [1e+00, 1e+01]
  Objective range [1e+00, 1e+00]
  Bounds range    [1e+00, 1e+00]
  RHS range       [1e+00, 1e+02]
Presolve removed 578 rows and 15 columns
Presolve time: 0.05s
Presolved: 5518 rows, 4426 columns, 22838 nonzeros
Variable types: 0 continuous, 4426 integer (4425 binary)

Root relaxation: objective 3.340000e+01, 1281 iterations, 0.05 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   33.40000    0   34          -   33.40000      -     -    0s
H    0     0                      28.0000000   33.40000  19.3%     -    0s
     0     0   33.00000    0  144   28.00000   33.00000  17.9%     -    0s
H    0     0                      29.0000000   33.00000  13.8%     -    0s
     0     0   32.84746    0  233   29.00000   32.84746  13.3%     -    0s
     0    

## Imprimindo solução no formato para avaliação

In [17]:
for var in [x for x in tuplelist (var_y).select () if var_y[x].getAttr ('X') == 1.0]:
    #print str(var)+"="+ str(1)
    pass

list =  [x for x in tuplelist (var_y).select () if var_y[x].getAttr ('X') == 1.0]

list = sorted (list, key=lambda tuple: tuple[2] )


with open('out.opt','a') as f:
    for var in list:
        f.write ("%s - %s:%s;\n" % (var[0],var[1],var[2]))
